In [1]:
import torch.nn.functional as F
import math
import torch
import torch.nn as nn
import torch.nn.parameter as Parameter


class SignumActivation(torch.autograd.Function):
    def forward(self, input):
        self.save_for_backward(input)
        size = input.size()
        mean = torch.mean(input.abs(), 1, keepdim=True)
        input = input.sign().add(0.01).sign()
        return input, mean

    def backward(self, grad_output, grad_output_mean): #STE Part
        input, = self.saved_tensors
        grad_input = grad_output.clone()
        grad_input=(2/torch.cosh(input))*(2/torch.cosh(input))*(grad_input)
        #grad_input[input.ge(1)] = 0 #great or equal
        #grad_input[input.le(-1)] = 0 #less or equal
        return grad_input

    
class BinarizeLinear(nn.Linear):
    def __init__(self, *kargs, **kwargs):
        super(BinarizeLinear, self).__init__(*kargs, **kwargs)

    def forward(self, input):
        #if input.size(1) != 784:
        #    input.data=input.data.sign().add(0.01).sign()
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=self.weight.org.sign().add(0.01).sign()
        out = nn.functional.linear(input, self.weight)
        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1).expand_as(out)
        return out


class BinConv2d(nn.Conv2d):
    def __init__(self, *kargs, **kwargs):
        super(BinConv2d, self).__init__(*kargs, **kwargs)

    def forward(self, input):
        #if input.size(1) != 3:
        #    input.data = input.data.sign().add(0.01).sign()
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=self.weight.org.sign().add(0.01).sign()
        out = nn.functional.conv2d(input, self.weight, None, self.stride,
                                   self.padding, self.dilation, self.groups)
        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1, 1, 1).expand_as(out)
        return out
      

class Unit_BinarizedConvolution2D(nn.Module):
    def __init__(self, input_channels, output_channels,
            kernel_size=-1, stride=-1, padding=-1, dropout=0, ActivationLayer=1, BNaffine=False):
        super(Unit_BinarizedConvolution2D, self).__init__()
        self.layer_type = 'BinConv2d'
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dropout_ratio = dropout
        if dropout!=0:
            self.dropout = nn.Dropout(dropout)
        self.ActivationLayer = ActivationLayer
        self.bn = nn.BatchNorm2d(output_channels, eps=1e-7, momentum=0.1, affine=BNaffine)
        if BNaffine==True:
            self.bn.weight.data = self.bn.weight.data.zero_().add(1.0)
        self.conv=BinConv2d(input_channels, output_channels,
                kernel_size=kernel_size, stride=stride, padding=padding)   #bias=False
    def forward(self, x):
       if self.dropout_ratio!=0:
            x = self.dropout(x)
       x = self.conv(x)
       x = self.bn(x)
       if self.ActivationLayer==1:
            x, mean = SignumActivation()(x)
       return x
    

class SignumActivationLayer(nn.Module):
    def forward(self, x):
        x, mean = SignumActivation()(x)
        return x
    
    

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.xnor = nn.Sequential(
        Unit_BinarizedConvolution2D(3, 64, kernel_size=3, stride=1, padding=1, ActivationLayer=1,BNaffine=False),
        Unit_BinarizedConvolution2D(64, 64, kernel_size=3, stride=1, padding=1, ActivationLayer=1,BNaffine=False),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=1),

        Unit_BinarizedConvolution2D(64, 128, kernel_size=3, stride=1, padding=1, ActivationLayer=1,BNaffine=False),
        Unit_BinarizedConvolution2D(128, 128, kernel_size=3, stride=1, padding=1, ActivationLayer=1,BNaffine=False),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            
        Unit_BinarizedConvolution2D(128, 256, kernel_size=3, stride=1, padding=1, ActivationLayer=1,BNaffine=False),
        Unit_BinarizedConvolution2D(256, 256, kernel_size=3, stride=1, padding=1, ActivationLayer=1,BNaffine=False),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
        nn.AvgPool2d(kernel_size=4, stride=2, padding=0),
        SignumActivationLayer()
        )
        
        self.classifier = nn.Sequential(
            BinarizeLinear(256, 10, bias=False),
            nn.BatchNorm1d(10, affine=False),
            nn.Softmax()
        )
    def forward(self, x):
        #for m in self.modules():
        #    if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
        #        if hasattr(m.weight, 'data'):
        #            m.weight.data.clamp_(min=0.01)
        x = self.xnor(x)
        #x = x.view(x.size(0), 256)
        x = x.view(x.size(0), -1)
        #x = x.view(-1, 256)
        x = self.classifier(x)
        return x
      
      

In [3]:
import time
import random
import matplotlib.pyplot as plt
import torch.utils.data as D
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets,transforms
import argparse



In [4]:
def timeSince(since):
    now = time.time()
    s = now - since
    #m = math.floor(s / 60)
    #s -= m * 60
    return s

  

In [5]:
parser = argparse.ArgumentParser(description='CIFAR Binarized weights')
parser.add_argument('--batch-size', type=int, default=50, metavar='N',help='input batch size , default =64')
parser.add_argument('--test-batch-size', type=int, default=50, metavar='N',help='input batch size for testing default=64')
parser.add_argument('--epochs', type=int, default=400, metavar='N',help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR',help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',help='random seed,default=1)')
parser.add_argument('--eps', type=float, default=1e-5, metavar='LR',help='learning rate,default=1e-5')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',help='for printing  training data is log interval')
parser.add_argument('--best_acc', type=float, default=0, metavar='N',help='Record of best accuracy')


_StoreAction(option_strings=['--best_acc'], dest='best_acc', nargs=None, const=None, default=0, type=<class 'float'>, choices=None, help='Record of best accuracy', metavar='N')

In [6]:
#args = parser.parse_args()
#args.cuda = not args.no_cuda and torch.cuda.is_available()


In [7]:

#torch.manual_seed(args.seed)
#if args.cuda:
#    torch.cuda.manual_seed(args.seed)
torch.cuda.manual_seed(1)
    #train_loader
train_loader = D.DataLoader(datasets.CIFAR10('./data', train=True, download=True,
                                             transform=transforms.Compose([transforms.ToTensor()
                                                                           #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                                                          ])) #transforms.RandomHorizontalFlip()
                            ,batch_size=10, shuffle=True) #500->args.test_batch_size
                                                 #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    
    #test_loaer
test_loader = D.DataLoader(datasets.CIFAR10('./data', train=False, 
                                            transform=transforms.Compose([transforms.ToTensor() #transforms.ToPILImage()
                                                                         
                                                                         ]))
                           ,batch_size=10, shuffle=False) #500->args.test_batch_size
 #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

Files already downloaded and verified


In [8]:
model=Net()
model.cuda()
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001) #args.momentum
#optimizer = optim.SGD(model.parameters(), lr=0.1,momentum=0.9) #args.momentum


In [9]:
def adjust_learning_rate(optimizer, epoch):
    update_list = [500, 600,700,800,900]
    if epoch in update_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
    return

In [10]:
def train(epoch):
    #global best_acc
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #if args.cuda:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        optimizer.zero_grad()
    
        loss.backward()
        
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
                
         
        optimizer.step()
        
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-1,1))
        
                

        if batch_idx % 100 == 0: #100-> args.log_interval
            tlos.append(loss.data)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))


In [11]:
def test():
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    acc = 100. * correct / len(test_loader)
    if acc > best_acc: #args.best_acc
        best_acc = acc
        #save_state(model, best_acc)
    test_loss /= len(test_loader) 
       
    accur.append( 100.*correct/total)
    print(batch_idx, len(test_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print('Best Accuracy:: ',best_acc)

In [12]:
start = time.time()
time_graph=[]
e=[]
accur=[]
tlos=[]
best_acc=0

In [ ]:
for epoch in range(1, 900 + 1): #600 -> args.epochs
    adjust_learning_rate(optimizer, epoch)
    e.append(epoch)
    train(epoch)   
    seco=timeSince(start)
    time_graph.append(seco)
    test()

print(time_graph)
plt.title('Training for CIFAR10 with epoch', fontsize=20)
plt.ylabel('time (s)')
plt.plot(e,time_graph)
plt.show()
plt.title('Accuracy With epoch', fontsize=20)
plt.plot(e,accur)
plt.show()
plt.title('Test loss With epoch', fontsize=20)
plt.plot(tlos)
plt.show()

C:\Users\Jeonghoon\.conda\envs\Lab4_Python37\lib\site-packages\torch\nn\modules\container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.285349
Train Epoch: 1 [1000/50000 (2%)]	Loss: 2.312907
Train Epoch: 1 [2000/50000 (4%)]	Loss: 2.252303
Train Epoch: 1 [3000/50000 (6%)]	Loss: 2.226883
Train Epoch: 1 [4000/50000 (8%)]	Loss: 2.235201
Train Epoch: 1 [5000/50000 (10%)]	Loss: 2.237994
Train Epoch: 1 [6000/50000 (12%)]	Loss: 2.246626
Train Epoch: 1 [7000/50000 (14%)]	Loss: 2.222377
Train Epoch: 1 [8000/50000 (16%)]	Loss: 2.255702
Train Epoch: 1 [9000/50000 (18%)]	Loss: 2.261592
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.149219
Train Epoch: 1 [11000/50000 (22%)]	Loss: 2.230226
Train Epoch: 1 [12000/50000 (24%)]	Loss: 2.235239
Train Epoch: 1 [13000/50000 (26%)]	Loss: 2.240024
Train Epoch: 1 [14000/50000 (28%)]	Loss: 2.287393
Train Epoch: 1 [15000/50000 (30%)]	Loss: 2.167699
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.192484
Train Epoch: 1 [17000/50000 (34%)]	Loss: 2.204805
Train Epoch: 1 [18000/50000 (36%)]	Loss: 2.269165
Train Epoch: 1 [19000/50000 (38%)]	Loss: 2.231339
Train Epoch: 1 [20

Train Epoch: 4 [11000/50000 (22%)]	Loss: 2.078550
Train Epoch: 4 [12000/50000 (24%)]	Loss: 2.046760
Train Epoch: 4 [13000/50000 (26%)]	Loss: 2.130100
Train Epoch: 4 [14000/50000 (28%)]	Loss: 2.095047
Train Epoch: 4 [15000/50000 (30%)]	Loss: 2.225964
Train Epoch: 4 [16000/50000 (32%)]	Loss: 2.199242
Train Epoch: 4 [17000/50000 (34%)]	Loss: 2.099835
Train Epoch: 4 [18000/50000 (36%)]	Loss: 2.185758
Train Epoch: 4 [19000/50000 (38%)]	Loss: 2.062935
Train Epoch: 4 [20000/50000 (40%)]	Loss: 2.087598
Train Epoch: 4 [21000/50000 (42%)]	Loss: 1.998656
Train Epoch: 4 [22000/50000 (44%)]	Loss: 2.169436
Train Epoch: 4 [23000/50000 (46%)]	Loss: 2.228685
Train Epoch: 4 [24000/50000 (48%)]	Loss: 2.141074
Train Epoch: 4 [25000/50000 (50%)]	Loss: 2.031339
Train Epoch: 4 [26000/50000 (52%)]	Loss: 2.066561
Train Epoch: 4 [27000/50000 (54%)]	Loss: 2.081097
Train Epoch: 4 [28000/50000 (56%)]	Loss: 2.115206
Train Epoch: 4 [29000/50000 (58%)]	Loss: 2.103060
Train Epoch: 4 [30000/50000 (60%)]	Loss: 2.047595


Train Epoch: 7 [22000/50000 (44%)]	Loss: 1.975777
Train Epoch: 7 [23000/50000 (46%)]	Loss: 2.056721
Train Epoch: 7 [24000/50000 (48%)]	Loss: 2.199806
Train Epoch: 7 [25000/50000 (50%)]	Loss: 2.143452
Train Epoch: 7 [26000/50000 (52%)]	Loss: 2.147010
Train Epoch: 7 [27000/50000 (54%)]	Loss: 2.077358
Train Epoch: 7 [28000/50000 (56%)]	Loss: 2.035399
Train Epoch: 7 [29000/50000 (58%)]	Loss: 2.036632
Train Epoch: 7 [30000/50000 (60%)]	Loss: 1.997709
Train Epoch: 7 [31000/50000 (62%)]	Loss: 2.002944
Train Epoch: 7 [32000/50000 (64%)]	Loss: 2.115154
Train Epoch: 7 [33000/50000 (66%)]	Loss: 2.158915
Train Epoch: 7 [34000/50000 (68%)]	Loss: 1.930774
Train Epoch: 7 [35000/50000 (70%)]	Loss: 2.060285
Train Epoch: 7 [36000/50000 (72%)]	Loss: 2.132470
Train Epoch: 7 [37000/50000 (74%)]	Loss: 2.131670
Train Epoch: 7 [38000/50000 (76%)]	Loss: 2.237633
Train Epoch: 7 [39000/50000 (78%)]	Loss: 2.117121
Train Epoch: 7 [40000/50000 (80%)]	Loss: 2.192479
Train Epoch: 7 [41000/50000 (82%)]	Loss: 2.028411


Train Epoch: 10 [32000/50000 (64%)]	Loss: 2.048010
Train Epoch: 10 [33000/50000 (66%)]	Loss: 2.144118
Train Epoch: 10 [34000/50000 (68%)]	Loss: 2.126026
Train Epoch: 10 [35000/50000 (70%)]	Loss: 2.048171
Train Epoch: 10 [36000/50000 (72%)]	Loss: 2.021837
Train Epoch: 10 [37000/50000 (74%)]	Loss: 2.050008
Train Epoch: 10 [38000/50000 (76%)]	Loss: 2.019095
Train Epoch: 10 [39000/50000 (78%)]	Loss: 2.061908
Train Epoch: 10 [40000/50000 (80%)]	Loss: 2.151380
Train Epoch: 10 [41000/50000 (82%)]	Loss: 1.894930
Train Epoch: 10 [42000/50000 (84%)]	Loss: 2.222265
Train Epoch: 10 [43000/50000 (86%)]	Loss: 2.012494
Train Epoch: 10 [44000/50000 (88%)]	Loss: 2.161388
Train Epoch: 10 [45000/50000 (90%)]	Loss: 1.931002
Train Epoch: 10 [46000/50000 (92%)]	Loss: 2.026451
Train Epoch: 10 [47000/50000 (94%)]	Loss: 1.930586
Train Epoch: 10 [48000/50000 (96%)]	Loss: 2.055797
Train Epoch: 10 [49000/50000 (98%)]	Loss: 2.160300
999 1000 Loss: 0.002 | Acc: 51.230% (5123/10000)
Best Accuracy::  552.0
Train Epoc

...

Train Epoch: 42 [10000/50000 (20%)]	Loss: 2.034736
Train Epoch: 42 [11000/50000 (22%)]	Loss: 2.139073
Train Epoch: 42 [12000/50000 (24%)]	Loss: 1.978578
Train Epoch: 42 [13000/50000 (26%)]	Loss: 2.109341
Train Epoch: 42 [14000/50000 (28%)]	Loss: 1.960594
Train Epoch: 42 [15000/50000 (30%)]	Loss: 2.078545
Train Epoch: 42 [16000/50000 (32%)]	Loss: 2.048282
Train Epoch: 42 [17000/50000 (34%)]	Loss: 2.053797
Train Epoch: 42 [18000/50000 (36%)]	Loss: 1.948036
Train Epoch: 42 [19000/50000 (38%)]	Loss: 2.122424
Train Epoch: 42 [20000/50000 (40%)]	Loss: 2.020023
Train Epoch: 42 [21000/50000 (42%)]	Loss: 1.988215
Train Epoch: 42 [22000/50000 (44%)]	Loss: 2.035734
Train Epoch: 42 [23000/50000 (46%)]	Loss: 1.993623
Train Epoch: 42 [24000/50000 (48%)]	Loss: 1.872363
Train Epoch: 42 [25000/50000 (50%)]	Loss: 1.963300
Train Epoch: 42 [26000/50000 (52%)]	Loss: 2.129855
Train Epoch: 42 [27000/50000 (54%)]	Loss: 2.274083
Train Epoch: 42 [28000/50000 (56%)]	Loss: 2.093184
Train Epoch: 42 [29000/50000 (5

Train Epoch: 45 [18000/50000 (36%)]	Loss: 1.994069
Train Epoch: 45 [19000/50000 (38%)]	Loss: 1.953159
Train Epoch: 45 [20000/50000 (40%)]	Loss: 2.077310
Train Epoch: 45 [21000/50000 (42%)]	Loss: 2.028495
Train Epoch: 45 [22000/50000 (44%)]	Loss: 2.086895
Train Epoch: 45 [23000/50000 (46%)]	Loss: 2.075804
Train Epoch: 45 [24000/50000 (48%)]	Loss: 2.039119
Train Epoch: 45 [25000/50000 (50%)]	Loss: 2.041437
Train Epoch: 45 [26000/50000 (52%)]	Loss: 2.202514
Train Epoch: 45 [27000/50000 (54%)]	Loss: 2.070836
Train Epoch: 45 [28000/50000 (56%)]	Loss: 2.038800
Train Epoch: 45 [29000/50000 (58%)]	Loss: 2.084570
Train Epoch: 45 [30000/50000 (60%)]	Loss: 2.095773
Train Epoch: 45 [31000/50000 (62%)]	Loss: 2.008903
Train Epoch: 45 [32000/50000 (64%)]	Loss: 1.927636
Train Epoch: 45 [33000/50000 (66%)]	Loss: 1.973510
Train Epoch: 45 [34000/50000 (68%)]	Loss: 1.973686
Train Epoch: 45 [35000/50000 (70%)]	Loss: 2.067382
Train Epoch: 45 [36000/50000 (72%)]	Loss: 2.076973
Train Epoch: 45 [37000/50000 (7

Train Epoch: 48 [26000/50000 (52%)]	Loss: 2.035244
Train Epoch: 48 [27000/50000 (54%)]	Loss: 1.990438
Train Epoch: 48 [28000/50000 (56%)]	Loss: 2.108314
Train Epoch: 48 [29000/50000 (58%)]	Loss: 2.067909
Train Epoch: 48 [30000/50000 (60%)]	Loss: 2.053770
Train Epoch: 48 [31000/50000 (62%)]	Loss: 2.020525
Train Epoch: 48 [32000/50000 (64%)]	Loss: 2.127036
Train Epoch: 48 [33000/50000 (66%)]	Loss: 2.071722
Train Epoch: 48 [34000/50000 (68%)]	Loss: 2.081462
Train Epoch: 48 [35000/50000 (70%)]	Loss: 2.123200
Train Epoch: 48 [36000/50000 (72%)]	Loss: 2.068047
Train Epoch: 48 [37000/50000 (74%)]	Loss: 1.978175
Train Epoch: 48 [38000/50000 (76%)]	Loss: 2.011756
Train Epoch: 48 [39000/50000 (78%)]	Loss: 2.114161
Train Epoch: 48 [40000/50000 (80%)]	Loss: 2.035908
Train Epoch: 48 [41000/50000 (82%)]	Loss: 2.003191
Train Epoch: 48 [42000/50000 (84%)]	Loss: 1.973214
Train Epoch: 48 [43000/50000 (86%)]	Loss: 2.014661
Train Epoch: 48 [44000/50000 (88%)]	Loss: 1.985975
Train Epoch: 48 [45000/50000 (9

Train Epoch: 51 [34000/50000 (68%)]	Loss: 2.048216
Train Epoch: 51 [35000/50000 (70%)]	Loss: 2.036468
Train Epoch: 51 [36000/50000 (72%)]	Loss: 2.125130
Train Epoch: 51 [37000/50000 (74%)]	Loss: 2.074396
Train Epoch: 51 [38000/50000 (76%)]	Loss: 2.004265
Train Epoch: 51 [39000/50000 (78%)]	Loss: 2.197465
Train Epoch: 51 [40000/50000 (80%)]	Loss: 2.019317
Train Epoch: 51 [41000/50000 (82%)]	Loss: 1.979792
Train Epoch: 51 [42000/50000 (84%)]	Loss: 2.003400
Train Epoch: 51 [43000/50000 (86%)]	Loss: 2.145022
Train Epoch: 51 [44000/50000 (88%)]	Loss: 2.041792
Train Epoch: 51 [45000/50000 (90%)]	Loss: 2.024345
Train Epoch: 51 [46000/50000 (92%)]	Loss: 2.041532
Train Epoch: 51 [47000/50000 (94%)]	Loss: 2.045066
Train Epoch: 51 [48000/50000 (96%)]	Loss: 1.980391
Train Epoch: 51 [49000/50000 (98%)]	Loss: 2.010039
999 1000 Loss: 0.002 | Acc: 61.170% (6117/10000)
Best Accuracy::  647.5
Train Epoch: 52 [0/50000 (0%)]	Loss: 2.008925
Train Epoch: 52 [1000/50000 (2%)]	Loss: 2.030583
Train Epoch: 52 [

Train Epoch: 54 [42000/50000 (84%)]	Loss: 2.106658
Train Epoch: 54 [43000/50000 (86%)]	Loss: 2.019491
Train Epoch: 54 [44000/50000 (88%)]	Loss: 1.987113
Train Epoch: 54 [45000/50000 (90%)]	Loss: 1.890870
Train Epoch: 54 [46000/50000 (92%)]	Loss: 2.029275
Train Epoch: 54 [47000/50000 (94%)]	Loss: 1.938171
Train Epoch: 54 [48000/50000 (96%)]	Loss: 2.033046
Train Epoch: 54 [49000/50000 (98%)]	Loss: 1.949793
999 1000 Loss: 0.002 | Acc: 61.490% (6149/10000)
Best Accuracy::  647.5
Train Epoch: 55 [0/50000 (0%)]	Loss: 2.071091
Train Epoch: 55 [1000/50000 (2%)]	Loss: 2.073386
Train Epoch: 55 [2000/50000 (4%)]	Loss: 1.994557
Train Epoch: 55 [3000/50000 (6%)]	Loss: 2.046383
Train Epoch: 55 [4000/50000 (8%)]	Loss: 1.956272
Train Epoch: 55 [5000/50000 (10%)]	Loss: 1.956340
Train Epoch: 55 [6000/50000 (12%)]	Loss: 1.964401
Train Epoch: 55 [7000/50000 (14%)]	Loss: 1.947507
Train Epoch: 55 [8000/50000 (16%)]	Loss: 2.015822
Train Epoch: 55 [9000/50000 (18%)]	Loss: 2.005386
Train Epoch: 55 [10000/50000